In [ ]:
import os
import sys
import glob
import json
import pickle

In [ ]:
DESCR_UNITS = 'descr/tft-champion.json'
DESCR_ITEMS = 'descr/tft-item.json'
DESCR_TRAITS = 'descr/tft-trait.json'
DESCR_AUGMENTS = 'descr/tft-augments.json'

MATCH_DIR = 'data/match/'
PLAYER_DIR = 'data/player/'

STATS_DIR = 'stats/'

In [ ]:
def stats_path(*names):
    assert len(names) > 0
    if len(names) == 1 and isinstance(names[0], tuple):
        names = names[0]
    names = list(names)
    names[-1] = f'{names[-1]}.txt'
    os.makedirs(os.path.join(STATS_DIR, *names[:-1]), exist_ok=True)
    return os.path.join(STATS_DIR, *names)

In [ ]:
unit_id_name = {}
item_id_name = {}
trait_id_name = {}
augment_id_name = {}

units_list = []
items_list = []
traits_list = []
augments_list = []

path_id_name_list_triples = [
    (DESCR_UNITS, unit_id_name, units_list),
    (DESCR_ITEMS, item_id_name, items_list),
    (DESCR_TRAITS, trait_id_name, traits_list),
    (DESCR_AUGMENTS, augment_id_name, augments_list)
]

for path, id_name, _ in path_id_name_list_triples:
    with open(path, 'r') as file:
        descrs = json.load(file)
    for descr_data in descrs['data'].values():
        name = descr_data['name'].removesuffix('<br>')
        if name == 'Ryze':
            namesuf = descr_data['id'].removeprefix('TFT9_Ryze')
            if namesuf != '':
                name += ' ' + namesuf
        if name == '':
            name = descr_data['id']
        id_name[descr_data['id']] = name

unit_id_name['TFT9_THex'] = 'THex'
unit_id_name['TFT9_HeimerdingerTurret'] = 'Heimerdinger Turret'

for _, id_name, name_list in path_id_name_list_triples:
    id_name_items = list(id_name.items())
    for iid, name in id_name_items:
        if name not in name_list:
            name_list.append(name)
        id_name[str.upper(iid)] = name
        id_name[str.lower(iid)] = name

In [ ]:
def sanitize_board(board):
    augments = [
        augment_id_name[augment]
        for augment in board['augments']
    ]

    traits = {
        trait_id_name[trait['name']] : trait['tier_current']
        for trait in board['traits'] if trait['tier_current'] > 0
    }

    def sanitize_items(items_raw):
        items = [item_id_name[item] for item in items_raw]
        tg_items = ['Thief\'s Gloves', 'Rascal\'s Gloves', 'Blacksmith\'s Gloves']
        if len(items) > 0 and items[0] in tg_items:
            return [items[0]]
        if 'TFT_Item_EmptyBag' in items:
            print(items_raw)
        return items

    units = [ {
            'name' : unit_id_name[unit['character_id']],
            'tier' : unit['tier'],
            'items' : sanitize_items(unit['itemNames'])
        } for unit in board['units']
    ]

    return {
        'puuid' : board['puuid'],
        'placement' : board['placement'],
        'level' : board['level'],
        'augments' : augments,
        'traits' : traits,
        'units' : units
    }

In [ ]:
def load_boards():
    all_boards_path = os.path.join(STATS_DIR, 'all_boards.pkl')

    if os.path.exists(all_boards_path):
        with open(all_boards_path, 'rb') as file:
            return pickle.load(file)

    boards = []

    for path in glob.glob(os.path.join(MATCH_DIR, '*.pkl')):
        with open(path, 'rb') as file:
            match = pickle.load(file)
        if match is None:
            continue
        if match['info']['queue_id'] != 1100:
            continue
        for board in match['info']['participants']:
            boards.append(sanitize_board(board))

    with open(all_boards_path, 'wb') as file:
        pickle.dump(boards, file)

    return boards

boards = load_boards()

print('Num boards:', len(boards))

In [ ]:
def empty_cond(board, *args):
    return True,

In [ ]:
def make_and_cond(*conds):
    def and_cond(board, *args):
        if len(conds) == 0:
            return True,
        for cond in conds[: -1]:
            val, *args = cond(board, *args)
            if not val:
                return False,
        return conds[-1](board, *args)
    return and_cond

In [ ]:
def compute_conditional_deltas(variable=empty_cond, conditional=empty_cond, relative_to_false=True):
    placements_dict = {}
    all_placements = []

    for board in boards:
        cond_val, *args = conditional(board)
        if not cond_val:
            continue

        placement = board['placement']

        all_placements.append(placement)

        var_vals, *args = variable(board, *args)

        if not isinstance(var_vals, list):
            var_vals = [var_vals]
        else:
            var_vals = list(set(var_vals))

        for var_val in var_vals:
            placements_dict.setdefault(var_val, []).append(placement)

    result_dict = {}

    for key, placements in placements_dict.items():
        expected = sum(placements) / len(placements)
        if relative_to_false:
            if len(placements) == len(all_placements):
                continue
            other_expected = (sum(all_placements) - sum(placements)) / (len(all_placements) - len(placements))
        else:
            other_expected = sum(all_placements) / len(all_placements)
        delta = expected - other_expected
        result_dict[key] = delta, len(placements)

    return result_dict

In [ ]:
def compute_conditional_delta(*args, **kwargs):
    return compute_conditional_deltas(*args, **kwargs).get(True, (0.0, 0))

In [ ]:
def compute_stats(name, options=None, make_option_cond=None, options_cond=None, conditional=empty_cond, min_cnt=50, relative_to_false=True):
    deltas_cnts = {}

    assert (options is None) == (make_option_cond is None)
    assert (options_cond is None) != (make_option_cond is None)

    if options_cond:
        deltas_cnts = compute_conditional_deltas(
            variable=options_cond,
            conditional=conditional,
            relative_to_false=relative_to_false)
    else:
        for option in options:
            deltas_cnts[option] = compute_conditional_delta(
                variable=make_option_cond(option),
                conditional=conditional,
                relative_to_false=relative_to_false)

    options_sorted = list(deltas_cnts.keys())
    options_sorted.sort(key=lambda option: deltas_cnts[option][0])

    max_len = max([0] + [len(str(option)) for option in options_sorted])

    if name is not None:
        file = open(stats_path(name), 'w')
    else:
        file = sys.stdout
    
    for option in options_sorted:
        delta, cnt = deltas_cnts[option]
        if cnt >= min_cnt:
            print(f'{str(option).ljust(max_len)}  {delta:+.2f}   {cnt:>6}', file=file)

    if name is not None:
        file.close

In [ ]:
def make_augment_cond(augment):
    def augment_cond(board, *args):
        return augment in board['augments'],
    return augment_cond

In [ ]:
def make_augments_cond():
    def augments_cond(board, *args):
        return board['augments'],
    return augments_cond

In [ ]:
def make_stage_augments_cond(stage):
    def augments_cond(board, *args):
        rev_idx = 4 - stage
        n = len(board['augments'])
        if n < rev_idx:
            return []
        return board['augments'][n - rev_idx],
    return augments_cond

In [ ]:
def make_unit_cond(unit_name, min_tier=1, min_items=0):
    def unit_cond(board, *args):
        for unit in board['units']:
            if unit['name'] == unit_name and unit['tier'] >= min_tier and len(unit['items']) >= min_items:
                return True, unit
        return False,
    return unit_cond

In [ ]:
def make_units_cond(min_tier=1, min_items=0):
    def units_cond(board, *args):
        unit_names = []
        for unit in board['units']:
            if unit['tier'] >= min_tier and len(unit['items']) >= min_items:
                unit_names.append(unit['name'])
        return unit_names,
    return units_cond

In [ ]:
def make_item_cond(item):
    def item_cond(board, unit, *args):
        return item in unit['items'], unit
    return item_cond

In [ ]:
def make_items_cond():
    def items_cond(board, unit, *args):
        return unit['items'],
    return items_cond

In [ ]:
def make_traits_cond(per_tier=True):
    def trait_cond(board, *args):
        if per_tier:
            return list(f'{trait} {tier}' for trait, tier in board['traits'].items()),
        else:
            return list(board['traits'].keys()),
    return trait_cond

In [ ]:
compute_stats(None, options_cond=make_stage_augments_cond())

In [ ]:
compute_stats('augments', options_cond=make_augments_cond())

In [ ]:
compute_stats('units', options_cond=make_units_cond())

In [ ]:
compute_stats('traits', options_cond=make_traits_cond())

In [ ]:
for unit_name in units_list:
    compute_stats(('per_unit', f'{unit_name} augments'), options_cond=make_augments_cond(), conditional=make_unit_cond(unit_name, min_items=3))

In [ ]:
for unit_name in units_list:
    compute_stats(('per_unit', f'{unit_name} items'), options_cond=make_items_cond(), conditional=make_unit_cond(unit_name, min_items=3))

In [ ]:
for unit_name in units_list:
    compute_stats(('per_unit', f'{unit_name} traits'), options_cond=make_traits_cond(), conditional=make_unit_cond(unit_name, min_items=3))